In [3]:
import numpy as np
import igl
import meshplot as mp
import math
import scipy.sparse as sp
from numpy import matlib as mb
from math import sqrt

# Load Data

In [34]:
v, f = igl.read_triangle_mesh('../data/cat.off')
v /= 10
mp.plot(v, f, shading={"point_size": 8})
kdmax, kdmin, kmax, kmin = igl.principal_curvature(v,f)  

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

In [50]:
def getAdjacentFaces(v,f):
    VF, NI = igl.vertex_triangle_adjacency(f, v.shape[0])
    VFi = []
    for i in range(NI.shape[0] - 1):
        VFii = []
        jj = NI[i + 1] - NI[i]
        for j in range(jj):
            VFii.append(VF[NI[i] + j])
        VFi.append(VFii)
    return VFi

VFi = getAdjacentFaces(v,f)               

# Find Regular Triangle

In [38]:
def sign(x):
    return (x > 0) - (x < 0)

def mark_regular(f,kdmax,kdmin):
    singular = []
    regular = []
    for indfi, fi in enumerate(f):
        sign_min = 1
        sign_max = 1
        for i in range(3):
            for j in range(i+1,3):
                sign_min *= np.dot(kdmin[fi[i]],kdmin[fi[j]])
                sign_max *= np.dot(kdmax[fi[i]],kdmax[fi[j]])
        if sign_min > 0 and sign_max > 0:
            regular.append(indfi)
        else:
            singular.append(indfi)
    return np.array(regular), np.array(singular)

regular, singular = mark_regular(f,kdmax,kdmin)

# Compute Area Star

In [17]:
def area_star(v, f):
    A = igl.doublearea(v,f)
    AS = np.zeros((v.shape[0],1))
    for indfi, fi in enumerate(f):
        for vi in fi:
            ind = vi
            AS[vi] += 0.5 * np.absolute(A[indfi])
    return AS

AS = area_star(v,f)

# Compute Extremalities

In [60]:
def extremalities(v, f, k, kd):
    EX = np.zeros((v.shape[0],1))
    A = igl.doublearea(v,f)
    G = igl.grad(v, f)
    grad_k = G * k
    neighbour = getAdjacentFaces(v,f) 
    for indvi, vi in enumerate(v):
        area = 0
        ex = 0
        for vfi in neighbour[indvi]:
            area += A[vfi] # compute area star
        for indvfi, vfi in enumerate(neighbour[indvi]):
            g = np.array([grad_k[vfi], grad_k[f.shape[0] + vfi], grad_k[2 * f.shape[0] + vfi]])
            ex += A[vfi] * np.dot(g,kd[indvi,:])
        ex /= area
        EX[indvi] = ex
    return EX

EX_max = extremalities(v, f, kmax, kdmax)
EX_min = extremalities(v, f, kmin, kdmin)
print(EX_max.shape)
print(f.shape)
print(v.shape)

(366, 1)
(698, 3)
(366, 3)


# Extract Feature Line

In [ ]:
def extract_feature_line(v, f, kmax, kmin, KD, EX, regular):
    # for regular faces
    fr = [f[index] for index in regular]
    TT, TTi = igl.triangle_triangle_adjacency(fr)
    edges = []
    marked_edges = []
    for indfri, fri in enumerate(fr):
        ex = [EX[index] for index in fri]
        kd = [KD[index] for index in fri]
        # flip signs
        for i in range(3):
            if(np.dot())
        

# Processing Singular Trangles

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120944…